In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [2]:
df = pd.read_csv('/home/breno-luiz/Documents/Planilha/caca.csv')
df.fillna(df.mean(), inplace=True)

X = df.drop('diagnosis', axis=1)
y = df['diagnosis']

In [3]:
minmax = MinMaxScaler()
X_norm = minmax.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=2/3, random_state=1)

In [4]:
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'max_leaf_nodes': [None, 10, 20]
}

param_grid_knn = {
    'n_neighbors': [3, 5, 7],
    'weights': ['uniform', 'distance']
}

param_grid_svc = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf']
}

In [5]:
grid_rf = GridSearchCV(RandomForestClassifier(), param_grid_rf, cv=5, scoring='accuracy')
grid_rf.fit(X_train, y_train)
best_rf = grid_rf.best_estimator_

grid_knn = GridSearchCV(KNeighborsClassifier(), param_grid_knn, cv=5, scoring='accuracy')
grid_knn.fit(X_train, y_train)
best_knn = grid_knn.best_estimator_

grid_svc = GridSearchCV(SVC(), param_grid_svc, cv=5, scoring='accuracy')
grid_svc.fit(X_train, y_train)
best_svc = grid_svc.best_estimator_

models = {'Random Forest': best_rf, 'KNN': best_knn, 'SVC': best_svc}

In [6]:
for name, model in models.items():
    y_pred = model.predict(X_test)
    print(f"{name} - Accuracy: {accuracy_score(y_test, y_pred):.4f}")
    print(f"{name} - Precision: {precision_score(y_test, y_pred, average='binary'):.4f}")
    print(f"{name} - Recall: {recall_score(y_test, y_pred, average='binary'):.4f}")
    print(f"{name} - F1 Score: {f1_score(y_test, y_pred, average='binary'):.4f}")
    print("\n")

print("Best Hyperparameters for Random Forest:", grid_rf.best_params_)
print("Best Hyperparameters for KNN:", grid_knn.best_params_)
print("Best Hyperparameters for SVC:", grid_svc.best_params_)

Random Forest - Accuracy: 0.8763
Random Forest - Precision: 0.9106
Random Forest - Recall: 0.8917
Random Forest - F1 Score: 0.9011


KNN - Accuracy: 0.8816
KNN - Precision: 0.9114
KNN - Recall: 0.9000
KNN - F1 Score: 0.9057


SVC - Accuracy: 0.8842
SVC - Precision: 0.9083
SVC - Recall: 0.9083
SVC - F1 Score: 0.9083


Best Hyperparameters for Random Forest: {'max_depth': 10, 'max_leaf_nodes': None, 'n_estimators': 50}
Best Hyperparameters for KNN: {'n_neighbors': 7, 'weights': 'uniform'}
Best Hyperparameters for SVC: {'C': 10, 'kernel': 'rbf'}
